##  Imports

In [1]:
import pandas as pd
import requests
import json


In [2]:
url_bitcoin = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=USD&days=30"
response_b = requests.get(url_bitcoin)
response_b
ds_b = pd.DataFrame(response_b.json())
ds_b.head()

,prices,market_caps,total_volumes
0,"[1564250554791, 9458.748968677053]","[1564250554791, 168769630193.2763]","[1564250554791, 21816510094.69829]"
1,"[1564254188554, 9416.385314494553]","[1564254188554, 167989628266.08728]","[1564254188554, 21832473518.718334]"
2,"[1564257787321, 9440.231281746921]","[1564257787321, 168550257864.4142]","[1564257787321, 21672627228.20599]"
3,"[1564261377679, 9471.833698643168]","[1564261377679, 168991868166.9875]","[1564261377679, 21539800749.68934]"
4,"[1564264826214, 9539.452875408946]","[1564264826214, 170199932354.29642]","[1564264826214, 21694699706.62936]"


In [18]:
#bitcoin = ds_b["prices"]
bitcoin = [x[1] for x in ds_b["prices"]]

bitcoin = pd.DataFrame(bitcoin)
bitcoin.columns = ["bitcoin_rate"]
bitcoin.head(10)

bitcoin["date"] = [str(pd.to_datetime(x[0], unit='ms'))[:11] for x in ds_b["prices"]]
bitcoin.head(10)

,bitcoin_rate,date
0,9458.748969,2019-07-27
1,9416.385314,2019-07-27
2,9440.231282,2019-07-27
3,9471.833699,2019-07-27
4,9539.452875,2019-07-27
5,9447.632727,2019-07-27
6,9480.330359,2019-07-28
7,9439.615244,2019-07-28
8,9512.812373,2019-07-28
9,9476.308954,2019-07-28


In [19]:
len(bitcoin["date"].unique().tolist())

31

In [20]:
bitcoin_final = bitcoin.groupby(["date"]).agg({'bitcoin_rate':'mean'}).reset_index()
bitcoin_final.head(10)

# And down is my two hours of my life
#bitcoin_final = pd.DataFrame(bitcoin_dates.index)
#bitcoin_final.head(10)
#bitcoin_final["bitcoin_mean_rate"] = bitcoin_dates["bitcoin_rate"]
#bitcoin_final.head(10)
#bitcoin_dates.columns
#bitcoin_dates
#n = bitcoin_dates["bitcoin_rate"]
#n = pd.DataFrame(bitcoin_dates, columns = ["bitcoin_mean_rate"])
#z = bitcoin_dates.to_dict("dict")
#k = pd.DataFrame(z)
#bitcoin_final["bitcoin_mean_rate"] = z["bitcoin_rate"]
#bitcoin_final
#k

,date,bitcoin_rate
0,2019-07-27,9462.380811
1,2019-07-28,9516.777434
2,2019-07-29,9556.353549
3,2019-07-30,9533.064726
4,2019-07-31,9830.911493
5,2019-08-01,10065.183360
6,2019-08-02,10446.422284
7,2019-08-03,10750.478958
8,2019-08-04,10777.797120
9,2019-08-05,11604.992007


# Make a dataframe with Bitcoin and Dogecoin market value for last 30 days vs USD

In [21]:
url_dogecoin = "https://api.coingecko.com/api/v3/coins/dogecoin/market_chart/range?vs_currency=usd&from=1564272000&to=1566826248%20"
response_d = requests.get(url_dogecoin)
response_d
ds_d = pd.DataFrame(response_d.json())
ds_d

ds_d.head()
#ds

,prices,market_caps,total_volumes
0,"[1564272022207, 0.0027998818906789015]","[1564272022207, 337453742.4224446]","[1564272022207, 42774398.98024047]"
1,"[1564275627270, 0.002786027658283239]","[1564275627270, 335802874.5619733]","[1564275627270, 42012954.28004315]"
2,"[1564279209364, 0.0027999051060619558]","[1564279209364, 337456540.439718]","[1564279209364, 42149760.190607116]"
3,"[1564282823437, 0.002788105313340157]","[1564282823437, 336065625.24412096]","[1564282823437, 42088681.31417209]"
4,"[1564286429393, 0.002791338220112297]","[1564286429393, 336946870.1147857]","[1564286429393, 42785057.16405686]"


In [22]:
dogecoin = [x[1] for x in ds_d["prices"]]
dogecoin = pd.DataFrame(dogecoin)
dogecoin.columns = ["dogecoin_rate"]
dogecoin.head(10)

dogecoin["date"] = [str(pd.to_datetime(x[0], unit='ms'))[:11] for x in ds_d["prices"]]
dogecoin.head(10)

,dogecoin_rate,date
0,0.002800,2019-07-28
1,0.002786,2019-07-28
2,0.002800,2019-07-28
3,0.002788,2019-07-28
4,0.002791,2019-07-28
5,0.002789,2019-07-28
6,0.002791,2019-07-28
7,0.002793,2019-07-28
8,0.002802,2019-07-28
9,0.002816,2019-07-28


In [28]:
len(dogecoin["date"].unique().tolist())

30

In [27]:
len(bitcoin["date"].unique().tolist())

31

In [26]:
dogecoin_final = dogecoin.groupby(["date"]).agg({'dogecoin_rate':'mean'}).reset_index()
bitcoin_final = bitcoin_final.drop(bitcoin_final.index[0])
#bitcoin_final = bitcoin_final.reset_index()
#bitcoin_final = bitcoin_final[["dogecoin_date", "bitcoin_rate"]]
#bitcoin_final.columns

In [11]:

print(dogecoin_final)
print(bitcoin_final)

   dogecoin_date  dogecoin_rate
0    2019-07-28        0.002806
1    2019-07-29        0.002816
2    2019-07-30        0.002790
3    2019-07-31        0.002833
4    2019-08-01        0.002863
5    2019-08-02        0.002900
6    2019-08-03        0.002928
7    2019-08-04        0.002912
8    2019-08-05        0.003036
9    2019-08-06        0.003004
10   2019-08-07        0.002952
11   2019-08-08        0.002946
12   2019-08-09        0.002901
13   2019-08-10        0.002871
14   2019-08-11        0.002877
15   2019-08-12        0.002879
16   2019-08-13        0.002849
17   2019-08-14        0.002783
18   2019-08-15        0.002648
19   2019-08-16        0.002630
20   2019-08-17        0.002651
21   2019-08-18        0.002670
22   2019-08-19        0.002776
23   2019-08-20        0.002766
24   2019-08-21        0.002660
25   2019-08-22        0.002653
26   2019-08-23        0.002684
27   2019-08-24        0.002666
28   2019-08-25        0.002654
29   2019-08-26        0.002656
   bitco

In [12]:
len(dogecoin_final["dogecoin_date"].unique().tolist())

30

In [13]:
len(bitcoin_final["bitcoin_date"].unique().tolist())

30

In [29]:
final = bitcoin_final.merge(dogecoin_final, on='date')
final

,date,bitcoin_rate,dogecoin_rate
0,2019-07-28,9516.777434,0.002806
1,2019-07-29,9556.353549,0.002816
2,2019-07-30,9533.064726,0.002790
3,2019-07-31,9830.911493,0.002833
4,2019-08-01,10065.183360,0.002863
5,2019-08-02,10446.422284,0.002900
6,2019-08-03,10750.478958,0.002928
7,2019-08-04,10777.797120,0.002912
8,2019-08-05,11604.992007,0.003036
9,2019-08-06,11749.143921,0.003004


# Top Ten Coins Report

In [30]:
url2 =  "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=10&page=1&sparkline=false"
response2 = requests.get(url2)
response2.headers

{'Date': 'Mon, 26 Aug 2019 18:03:56 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=dbf4bdb5df3b9b6acc22a23efd13c3d131566842635; expires=Tue, 25-Aug-20 18:03:55 GMT; path=/; domain=.coingecko.com; HttpOnly', 'Cache-Control': 'max-age=60, public, must-revalidate, s-maxage=60', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'POST, PUT, DELETE, GET, OPTIONS', 'Access-Control-Request-Method': '*', 'Access-Control-Allow-Headers': 'Origin, X-Requested-With, Content-Type, Accept, Authorization', 'Access-Control-Expose-Headers': 'link, per-page, total', 'Vary': 'Accept-Encoding, Origin', 'Content-Encoding': 'gzip', 'ETag': 'W/"b60cd133fdf680c0271d5f1182351db9"', 'X-Request-Id': '09918c4c-d3bf-437d-963c-dfdcc6b853b0', 'X-Runtime': '0.456435', 'Alternate-Protocol': '443:npn-spdy/2', 'CF-Cache-Status': 'EXPIRED', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.

## List the first 10 coingecko currencies in a DataFrame

In [31]:
ds = pd.DataFrame(response2.json())
ds.head()

,ath,ath_change_percentage,ath_date,circulating_supply,current_price,high_24h,id,image,last_updated,low_24h,...,market_cap_change_24h,market_cap_change_percentage_24h,market_cap_rank,name,price_change_24h,price_change_percentage_24h,roi,symbol,total_supply,total_volume
0,19665.39,-47.61936,2017-12-16T00:00:00.000Z,1.789834e+07,10298.080000,10673.830000,bitcoin,https://assets.coingecko.com/coins/images/1/la...,2019-08-26T18:03:17.599Z,9990.11000,...,3.496489e+09,1.93363,1,Bitcoin,201.340000,1.99413,None,btc,2.100000e+07,23154601152
1,1448.18,-87.02648,2018-01-13T00:00:00.000Z,1.074831e+08,187.840000,194.000000,ethereum,https://assets.coingecko.com/coins/images/279/...,2019-08-26T18:03:10.898Z,183.66000,...,5.636754e+07,0.27990,2,Ethereum,0.450272,0.24028,"{'times': 23.360792638641268, 'currency': 'btc...",eth,NaN,7515824146
2,3.40,-92.10696,2018-01-07T00:00:00.000Z,4.290954e+10,0.268125,0.278334,ripple,https://assets.coingecko.com/coins/images/44/l...,2019-08-26T18:03:39.188Z,0.26708,...,-1.041380e+08,-0.89691,3,XRP,-0.002465,-0.91079,None,xrp,1.000000e+11,1428738690
3,3785.82,-91.83581,2017-12-20T00:00:00.000Z,1.796888e+07,309.020000,314.640000,bitcoin-cash,https://assets.coingecko.com/coins/images/780/...,2019-08-26T18:03:41.809Z,301.84000,...,9.981717e+07,1.83048,4,Bitcoin Cash,5.520000,1.81720,None,bch,2.100000e+07,1587957199
4,360.66,-79.44520,2017-12-18T00:00:00.000Z,6.311849e+07,74.150000,74.990000,litecoin,https://assets.coingecko.com/coins/images/2/la...,2019-08-26T18:03:40.774Z,70.89000,...,1.027534e+08,2.24575,5,Litecoin,1.740000,2.40644,None,ltc,8.400000e+07,3129484481


In [32]:
ds_top = ds[['id','name','market_cap','market_cap_change_percentage_24h']].copy()
ds_top.head(10)

,id,name,market_cap,market_cap_change_percentage_24h
0,bitcoin,Bitcoin,184321347134,1.93363
1,ethereum,Ethereum,20194785541,0.27990
2,ripple,XRP,11506596122,-0.89691
3,bitcoin-cash,Bitcoin Cash,5552888799,1.83048
4,litecoin,Litecoin,4678221345,2.24575
5,binancecoin,Binance Coin,4015968955,-0.55781
6,eos,EOS,3656725611,1.03853
7,tether,Tether,3556747784,-0.06536
8,bitcoin-cash-sv,Bitcoin SV,2389296832,0.58827
9,cardano,Cardano,1552310884,-1.58516


## Make a csv called 'top_ten_report.csv' and check that it worked

In [33]:
ds_top.to_csv("top_ten.csv", index=False)